In [9]:
import cvxpy as cp
import numpy as np
from itertools import product

In [10]:
shipping_cost = np.array([[1,3,2],[3,2,2]]).reshape(2,3)
all_demand = np.array([[100,150,120],[120,180,150],[150,200,180]]).reshape(3,3)
selling_price = np.array([[16,16]]).reshape(2,1)
produce = np.array([100,200]).reshape(2,1)
production_cost = np.array([11,10]).reshape(2,1)

In [11]:
senario = np.array([[100,120,150],
       [150,180,200],[120,150,180]])
senario_comb = []
index = np.arange(0,3)
index_permute = product(index,repeat=3)
for i in index_permute:
    temp_array =[]
    for j in range(0,3):
        temp_array.append(float(senario[j,:][i[j]]))
    senario_comb.append(temp_array)
all_demand = np.array(senario_comb).reshape(27,3)

In [12]:
def solve_second_stage_problem(produce,all_demand):

    duals = []
    vs = []

    for demand in all_demand:
        
        sell = cp.Variable(shape=(2,3))
        salvage = cp.Variable(shape=(2,1))

        objective = cp.Minimize(-16*cp.sum(sell,keepdims=True) + cp.trace(((shipping_cost@(sell.T)))))
        constraints = [cp.sum(sell,axis=1,keepdims=True) +salvage <= produce,cp.sum(sell,axis=0,keepdims=True) <= demand.reshape(1,-1), sell>= np.zeros_like(sell),salvage>= np.zeros_like(salvage)]

        problem = cp.Problem(objective, constraints)
        problem.solve()

        dual_value = problem.constraints[0].dual_value

        duals.append(dual_value)
        vs.append(problem.value)
        
    return np.array(duals).reshape(-1,2),np.array(vs).reshape(-1,1)

In [13]:
duals,vs = solve_second_stage_problem(produce,all_demand)

In [14]:
senario = 3
senario_prob = (1/senario)*(np.ones_like(all_demand[:,0])).reshape(-1,1)


In [15]:
g_ks_temp = (-senario_prob.T@duals).T
alpha_ks_temp = senario_prob.T@vs - g_ks_temp.T@produce

In [16]:
def solve_first_stage_problem(produce,g_ks,alpha_ks,senario_prob): 

    size = 2
    production_cost = np.array([11,10]).reshape(2,1)

    produce  = cp.Variable(shape=(2,1))
    produce_limit = np.array([300,300]).reshape(2,1)

    v = cp.Variable(shape=(len(senario_prob),1))

    constraints = []
    for i in range(0,len(g_ks)):
        for j in range(0,len(g_ks[i])):
            constraints.append(np.array([g_ks[i][j]])@produce+np.array([alpha_ks[i][j]])<=v[j])

    constraints.extend([produce>=0,v>=-1000000,produce<=produce_limit])

    objective = cp.Minimize(production_cost.T@produce + senario_prob.T@v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    print(problem.value)

    return produce.value, v.value

In [17]:
# produce = np.array([3,3,3]).reshape(-1,1)
limit = -100000

g_ks = []
alpha_ks = []

senario = 27
senario_prob = (1/senario)*(np.ones_like(all_demand[:,0])).reshape(-1,1)

for i in range(0,300):
    duals,qs = solve_second_stage_problem(produce,all_demand)

    gks_batch = []
    alpha_ks_batch = []

    for i in range(0,senario):
        gks_batch.append(-duals[i])
        alpha_ks_batch.append(qs[i]+duals[i].T@produce)
  
    g_ks.append(gks_batch)
    alpha_ks.append(alpha_ks_batch)

    produce,limit= solve_first_stage_problem(produce,g_ks,alpha_ks,senario_prob)
    print(produce)
    
    # print(limit)

-2351.8959410451316
[[299.99999959]
 [299.99999969]]
-1612.1518386785974
[[120.00000007]
 [299.99999995]]
-1609.99999314676
[[120.00000112]
 [299.99999941]]
-1609.9999926396595
[[120.00000085]
 [299.99999957]]
-1609.9999910978004
[[120.00000145]
 [299.9999993 ]]
-1609.999991087253
[[120.00000161]
 [299.99999922]]
-1609.999985394923
[[120.00000058]
 [299.99999993]]
-1609.9999900562188
[[120.00000129]
 [299.9999991 ]]
-1609.9999911252935
[[120.00000193]
 [299.99999983]]
-1609.9999880200967
[[119.99999975]
 [299.99999995]]
-1609.9999893791382
[[120.00000164]
 [299.99999997]]
-1609.9999832847798
[[120.00000194]
 [299.99999985]]
-1609.9999854362813
[[120.00000066]
 [300.00000002]]
-1609.9999896250965
[[120.00000187]
 [299.99999993]]
-1609.999984013094
[[120.00000071]
 [300.        ]]
-1609.9999875280037
[[120.00000111]
 [300.        ]]
-1609.9999867544611
[[120.00000099]
 [300.00000001]]
-1609.999986278719
[[120.00000098]
 [300.00000003]]
-1609.9999876447628
[[120.00000131]
 [300.00000002]]

KeyboardInterrupt: 